Running importance analysis with Hail
=====================================

This is an *VariantSpark* example notebook.


One of the main applications of VariantSpark is discovery of genomic variants correlated with a response variable (e.g. case vs control) using random forest gini importance.

The `chr22_1000.vcf` is a very small sample of the chromosome 22 VCF file from the 1000 Genomes Project.

`chr22-labels-hail.csv` is a CSV file with sample response variables (labels). In fact the labels directly represent the number of alternative alleles for each sample at a specific genomic position. E.g.: column x22_16050408 has labels derived from variants in chromosome 22 position 16050408. We would expect then that position 22:16050408 in the VCF file is strongly correlated with the label x22_16050408.

Both data sets are located in the `..\data` directory.

This notebook demonstrates how to run importance analysis on these data with *VariantSpark* Hail integration.

Step 1: Create a `HailContext` using `SparkContext` object (here injected as `sc`):

In [0]:
import hail as hl
import varspark.hail as vshl
vshl.init(sc, idempotent=True)

using variant-spark jar at '/databricks/python/lib/python3.8/site-packages/varspark/jars/variant-spark_2.12-0.5.2-all.jar'
pip-installed Hail requires additional configuration options in Spark referring
 to the path to the Hail Python module directory HAIL_DIR,
 e.g. /path/to/python/site-packages/hail:
 spark.jars=HAIL_DIR/hail-all-spark.jar
 spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
 spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.1.2
SparkUI available at http://10.179.0.7:40001
Welcome to
 __ __ <>__
 / /_/ /__ __/ /
 / __ / _ `/ / /
 /_/ /_/\_,_/_/_/ version 0.2.74-0c3a74d12093
LOGGING: writing to /databricks/driver/hail-20221031-0640-0.2.74-0c3a74d12093.log

Step 2: Load Hail variant dataset  `vds` from a sample `.vcf` file.

In [0]:
vds = hl.import_vcf('dbfs:/databricks/Filestore/chr22_1000.vcf')

Step 3: Load labels into Hail table `labels`.

In [0]:
labels = hl.import_table('dbfs:/databricks/Filestore/chr22-labels-hail.csv', impute = True, delimiter=",").key_by('sample')

2022-10-31 06:40:52 Hail: WARN: Name collision: field 'sample' already in object dict. 
 This field must be referenced with __getitem__ syntax: obj['sample']
2022-10-31 06:40:52 Hail: INFO: Reading table to impute column types
2022-10-31 06:41:18 Hail: INFO: Finished type imputation
 Loading field 'sample' as type str (imputed)
 Loading field 'x22_16050408' as type int32 (imputed)
 Loading field 'x22_16050612' as type str (imputed)
 Loading field 'x22_16050678' as type str (imputed)
 Loading field 'x22_16050984' as type int32 (imputed)
 Loading field 'x22_16051107' as type int32 (imputed)
 Loading field 'x22_16051249' as type int32 (imputed)
 Loading field 'x22_16051347' as type int32 (imputed)
 Loading field 'x22_16051453' as type int32 (imputed)
 Loading field 'x22_16051477' as type int32 (imputed)
 Loading field 'x22_16051480' as type int32 (imputed)

Step 4: Annotate dataset samples with labels.

In [0]:
vds = vds.annotate_cols(label = labels[vds.s])
vds.cols().show(3)

2022-10-31 06:41:18 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
 To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2022-10-31 06:41:19 Hail: INFO: Coerced almost-sorted dataset
2022-10-31 06:41:55 Hail: INFO: Coerced sorted dataset
+-----------+--------------------+--------------------+--------------------+
 s | label.x22_16050408 | label.x22_16050612 | label.x22_16050678 |
+-----------+--------------------+--------------------+--------------------+
 str | int32 | str | str |
+-----------+--------------------+--------------------+--------------------+
 "HG00096" | 0 | "hahaha" | "heheh" |
 "HG00097" | 1 | "ala ma" | "1" |
 "HG00099" | 1 | "1" | "1" |
+-----------+--------------------+--------------------+--------------------+

+--------------------+--------------------+--------------------+
 label.x22_16050984 | label.x22_16051107 | label.x22_16051249 |
+--------------------+--------------------+--------------------+
 int32 | int32 | int32 |
+--------------------+--------------------+--------------------+
 0 | 0 | 0 |
 0 | 1 | 1 |
 0 | 1 | 1 |
+--------------------+--------------------+--------------------+

+--------------------+--------------------+--------------------+
 label.x22_16051347 | label.x22_16051453 | label.x22_16051477 |
+--------------------+--------------------+--------------------+
 int32 | int32 | int32 |
+--------------------+--------------------+--------------------+
 0 | 0 | 0 |
 1 | 1 | 0 |
 1 | 1 | 0 |
+--------------------+--------------------+--------------------+

+--------------------+
 label.x22_16051480 |
+--------------------+
 int32 |
+--------------------+
 0 |
 1 |
 1 |
+--------------------+
showing top 3 rows

Step 5: Build the random forest model with `label.x22_16050408` as the respose variable.

In [0]:
rf_model = vshl.random_forest_model(y=vds.label['x22_16050408'],
                x=vds.GT.n_alt_alleles(), seed = 13, mtry_fraction = 0.05, min_node_size = 5, max_depth = 10)
rf_model.fit_trees(100, 50)

2022-10-31 06:41:57 Hail: INFO: Coerced almost-sorted dataset

Step 6: Display the results: print OOB error calculated variable importance.

In [0]:
print("OOB error: %s" % rf_model.oob_error())
impTable = rf_model.variable_importance()
impTable.order_by(hl.desc(impTable.importance)).show(10)

OOB error: 0.007326007326007326
2022-10-31 06:42:10 Hail: INFO: Coerced sorted dataset
+---------------+------------+------------+------------+
 locus | alleles | importance | splitCount |
+---------------+------------+------------+------------+
 locus<GRCh37> | array<str> | float64 | int64 |
+---------------+------------+------------+------------+
 22:16050408 | ["T","C"] | 4.22e+01 | 73 |
 22:16050678 | ["C","T"] | 2.37e+01 | 38 |
 22:16052838 | ["T","A"] | 1.85e+01 | 24 |
 22:16053727 | ["T","G"] | 1.71e+01 | 17 |
 22:16051107 | ["C","A"] | 1.29e+01 | 23 |
 22:16053197 | ["G","T"] | 1.18e+01 | 15 |
 22:16051480 | ["T","C"] | 1.16e+01 | 25 |
 22:16053797 | ["T","C"] | 1.08e+01 | 13 |
 22:16051882 | ["C","T"] | 1.04e+01 | 26 |
 22:16052656 | ["T","C"] | 9.14e+00 | 16 |
+---------------+------------+------------+------------+
showing top 10 rows

Optionally release the resouces (RAM) associated with the model.

In [0]:
#rf_model.release()

For more information on using *VariantSpark* and the Python API and Hail integration please visit the [documentation](http://variantspark.readthedocs.io/en/latest/).